<a href="https://www.kaggle.com/code/stiwar1/tps-may22-data-preprocessing?scriptVersionId=95242244" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.max_columns', None)

from sklearn.model_selection import StratifiedKFold

SEED = 2311

In [2]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')

In [3]:
train.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.3732,0.2389,-0.2434,0.5674,-0.6477,0.8393,0.1131,1,5,1,3,3,3,1,6,1,0,7,4,0.2982,-0.9197,3.0585,-2.5407,0.7670,-2.7306,-0.2082,1.3634,ABABDADBAB,67.6092,0,0,0
1,1,1.6970,-1.7103,-2.2303,-0.5457,1.1132,-1.5522,0.4478,1,3,4,0,2,3,0,1,0,4,6,0,-3.1477,-1.0754,2.1791,2.2783,-0.6337,-1.2171,-3.7822,-0.0583,ACACCADCEB,377.0964,0,0,1
2,2,1.6817,0.6167,-1.0277,0.8105,-0.6091,0.1140,-0.7087,1,0,2,6,6,4,3,1,2,2,1,4,2.8207,-3.4853,-0.7842,-1.3858,-0.5206,-0.0091,2.7885,-3.7035,AAAEABCKAD,-195.5997,0,2,1
3,3,-0.1182,-0.5878,-0.8046,2.0868,0.3710,-0.1288,-0.2826,3,2,1,0,1,6,4,2,3,3,0,3,1.0811,-2.1002,-2.3438,0.5726,-1.6532,1.6860,-2.5331,-0.6086,BDBBAACBCB,210.8262,0,0,1
4,4,1.1485,-0.1766,-0.6649,-1.1013,0.4679,0.5001,0.4075,3,3,0,4,3,0,6,0,3,3,1,0,-0.1262,0.6050,1.1337,-3.9129,-1.4304,2.1276,-3.3068,4.3714,BDBCBBCHFE,-217.2118,0,1,1


In [4]:
#Missing values
train.isna().sum().sum(), test.isna().sum().sum()

(0, 0)

In [5]:
#Target class balance
train.target.value_counts(normalize=True)

0    0.5135
1    0.4865
Name: target, dtype: float64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      900000 non-null  int64  
 1   f_00    900000 non-null  float64
 2   f_01    900000 non-null  float64
 3   f_02    900000 non-null  float64
 4   f_03    900000 non-null  float64
 5   f_04    900000 non-null  float64
 6   f_05    900000 non-null  float64
 7   f_06    900000 non-null  float64
 8   f_07    900000 non-null  int64  
 9   f_08    900000 non-null  int64  
 10  f_09    900000 non-null  int64  
 11  f_10    900000 non-null  int64  
 12  f_11    900000 non-null  int64  
 13  f_12    900000 non-null  int64  
 14  f_13    900000 non-null  int64  
 15  f_14    900000 non-null  int64  
 16  f_15    900000 non-null  int64  
 17  f_16    900000 non-null  int64  
 18  f_17    900000 non-null  int64  
 19  f_18    900000 non-null  int64  
 20  f_19    900000 non-null  float64
 21  f_20    90

In [7]:
features = [f for f in train.columns if f not in ('id', 'target')]

num_features = train.select_dtypes('float64').columns.to_list()

In [8]:
def split_and_encode(df, feature='f_27'):
    '''
    Handles feature 'f_27' by splitting the string, 
    creating new columns for each character and then
    encoding them with integers. Also creates a column 
    for count of unique characters in feature 'f_27'.
    
    Each string in 'f_27' has exactly 10 uppercase 
    characters for both train and test sets.
    '''
    
    expanded_df = pd.DataFrame(
        df[feature].apply(list).to_list(), 
        columns=[(feature + '_' + str(i)) for i in range(10)])
    
    for col in expanded_df.columns:
        expanded_df[col] = expanded_df[col].apply(lambda x: ord(x) - ord('A'))
        
    expanded_df[feature + '_unique'] = df[feature].apply(set).apply(len)
    
    return expanded_df

In [9]:
train = pd.concat([train, split_and_encode(train, 'f_27')], axis=1)
test = pd.concat([test, split_and_encode(test, 'f_27')], axis=1)

In [10]:
train.drop(['f_27'], axis=1, inplace=True)
test.drop(['f_27'], axis=1, inplace=True)

In [11]:
#updating features list with created columns
features = [f for f in train.columns if f not in ('id', 'target')]
num_features.append('f_27_unique')
cat_features = [f for f in features if f not in num_features]

In [12]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

In [13]:
N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
train['fold'] = -1

for fold, (_, idx) in enumerate(skf.split(X=train, y=train['target'])):
    train.loc[idx, 'fold'] = fold

In [14]:
train.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30,target,f_27_0,f_27_1,f_27_2,f_27_3,f_27_4,f_27_5,f_27_6,f_27_7,f_27_8,f_27_9,f_27_unique,fold
0,0,-1.3732,0.2389,-0.2434,0.5674,-0.6477,0.8393,0.1131,1,5,1,3,3,3,1,6,1,0,7,4,0.2982,-0.9197,3.0585,-2.5407,0.7670,-2.7306,-0.2082,1.3634,67.6092,0,0,0,0,1,0,1,3,0,3,1,0,1,3,2
1,1,1.6970,-1.7103,-2.2303,-0.5457,1.1132,-1.5522,0.4478,1,3,4,0,2,3,0,1,0,4,6,0,-3.1477,-1.0754,2.1791,2.2783,-0.6337,-1.2171,-3.7822,-0.0583,377.0964,0,0,1,0,2,0,2,2,0,3,2,4,1,5,4
2,2,1.6817,0.6167,-1.0277,0.8105,-0.6091,0.1140,-0.7087,1,0,2,6,6,4,3,1,2,2,1,4,2.8207,-3.4853,-0.7842,-1.3858,-0.5206,-0.0091,2.7885,-3.7035,-195.5997,0,2,1,0,0,0,4,0,1,2,10,0,3,6,2
3,3,-0.1182,-0.5878,-0.8046,2.0868,0.3710,-0.1288,-0.2826,3,2,1,0,1,6,4,2,3,3,0,3,1.0811,-2.1002,-2.3438,0.5726,-1.6532,1.6860,-2.5331,-0.6086,210.8262,0,0,1,1,3,1,1,0,0,2,1,2,1,4,0
4,4,1.1485,-0.1766,-0.6649,-1.1013,0.4679,0.5001,0.4075,3,3,0,4,3,0,6,0,3,3,1,0,-0.1262,0.6050,1.1337,-3.9129,-1.4304,2.1276,-3.3068,4.3714,-217.2118,0,1,1,1,3,1,2,1,1,2,7,5,4,6,4


In [15]:
test.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30,f_27_0,f_27_1,f_27_2,f_27_3,f_27_4,f_27_5,f_27_6,f_27_7,f_27_8,f_27_9,f_27_unique
0,900000,0.4425,0.1744,-0.9998,0.7627,0.1868,-1.0748,0.5019,6,6,0,0,0,5,1,1,2,2,0,1,-1.1284,2.7493,-1.0064,-1.1939,-2.4357,-2.4274,-1.9669,5.7342,99.4784,0,0,1,0,0,0,1,0,3,11,0,2,5
1,900001,-0.6056,-0.3057,0.6277,-0.5789,-1.7509,1.3556,-0.1909,1,3,4,0,2,1,3,0,0,0,2,0,-4.4241,1.0808,2.3824,0.1494,1.8833,-2.8487,-0.7252,3.1942,-65.9938,1,0,0,5,0,1,1,0,4,6,2,1,6
2,900002,0.3040,2.4451,0.2465,0.8182,0.3597,-1.3318,1.3586,3,3,4,1,1,3,2,2,4,1,0,3,-1.5239,-1.4067,-7.0261,1.3123,-5.1572,1.7140,0.5850,0.0669,-87.4056,0,1,1,1,0,2,0,1,1,10,4,4,5
3,900003,0.1541,0.2601,-1.3671,-0.0932,-1.1110,-0.9485,1.1192,0,0,4,2,1,5,0,3,3,1,4,2,-1.4046,3.0111,-0.5945,-3.9395,1.7546,-2.3640,-1.0033,3.8931,-281.2935,0,0,0,4,1,4,0,0,2,16,2,2,5
4,900004,-1.6519,-0.4243,-0.6674,-0.3221,-0.0895,0.1817,1.7850,2,2,2,0,0,3,0,1,2,0,2,2,-1.9685,0.1006,0.0849,-0.9857,-0.1305,-3.5579,1.2107,1.8619,25.6294,0,2,0,4,1,1,1,1,3,0,1,5,5


In [16]:
train.to_csv('train_processed.csv', index=False)
test.to_csv('test_processed.csv', index=False)